In [1]:
from Model_Predict_Feature_Extraction import *

#from DLpTCR_server import *


import os
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.python.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.utils import plot_model
#from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint

from sklearn.utils import shuffle

from tensorflow.python.keras.models import load_model
import matplotlib.pyplot as plt
import csv
import pandas as pd

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 只显示error和warining信息 3 只显示error信息
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 这一行注释掉就是使用cpu，不注释就是使用gpu

import tensorflow.compat.v1 as tf

try:
    import tensorflow.python.keras as keras
except:
    import tensorflow.keras as keras

from tensorflow.python.keras import layers
from tensorflow.python.keras import backend as K

from sklearn.model_selection import KFold

from sklearn import metrics
from sklearn.metrics import accuracy_score,matthews_corrcoef,classification_report,confusion_matrix,precision_score,recall_score
from sklearn.metrics import f1_score,roc_auc_score, auc

from keras import regularizers

import os
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.python.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.utils import plot_model
#from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint

from sklearn.utils import shuffle

from tensorflow.python.keras.models import load_model
import matplotlib.pyplot as plt
import csv
import pandas as pd

    
def TCRB_Model_Integration(FULL_M,CNN_M,RESNET_M,FULL_Feature,CNN_Feature,RESNET_Feature):
    K.clear_session()
    tf.reset_default_graph()   
    
    FULL_model = load_model(FULL_M)
    CNN_model = load_model(CNN_M)
    RESNET_model = load_model(RESNET_M) 
    
    FULL_X = FULL_Feature
    FULL_X = FULL_X.reshape([len(FULL_X),20,19,2])

    CNN_X = CNN_Feature
    CNN_X = CNN_X.reshape([len(CNN_X),20,21,2])  
    
    RESNET_X = RESNET_Feature
    RESNET_X = RESNET_X.reshape([len(RESNET_X),20,11,2])

    Y_PRED_FULL = FULL_model.predict(FULL_X)
    Y_PRED_CNN = CNN_model.predict(CNN_X)
    Y_PRED_RESNET = RESNET_model.predict(RESNET_X)
    Y_pred_FULL=Y_PRED_FULL[:, 0]
    Y_PRED_CNN=Y_PRED_CNN[:, 0]
    Y_PRED_RESNET=Y_PRED_RESNET[:, 0]            

    del FULL_model
    del CNN_model
    del RESNET_model
    return Y_pred_FULL,Y_PRED_CNN,Y_PRED_RESNET
    
    
    
def pred_B(user_dir):
    TCRB_FULL_M = '../model/Article/FULL_B_ALL_pca18.h5'
    TCRB_CNN_M = '../model/Article/CNN_B_ALL_pca20.h5'
    TCRB_RESNET_M = '../model/Article/RESNET_B_ALL_pca10.h5'

    TCRB_FULL_Feature = np.load(str(user_dir) + "/TCRB_PCA18_feature_array.npy")
    TCRB_CNN_Feature= np.load(str(user_dir) + "/TCRB_PCA20_feature_array.npy")    
    TCRB_RESNET_Feature = np.load(str(user_dir) + "/TCRB_PCA10_feature_array.npy") 
    
    Y_pred_FULL,Y_PRED_CNN,Y_PRED_RESNET = TCRB_Model_Integration(TCRB_FULL_M,TCRB_CNN_M,TCRB_RESNET_M,
                                              TCRB_FULL_Feature,TCRB_CNN_Feature,TCRB_RESNET_Feature)
    
    return Y_pred_FULL,Y_PRED_CNN,Y_PRED_RESNET



def save_outputfile(user_dir,save_path,user_select,excel_file_path,TCRB_cdr3,Epitope,y_true):
    if user_select == 'A':
        print(user_select)

        TCRA_Y_pred_ALL,TCRA_Y_pred_ALL_avg = pred_inte_all(user_dir,user_select)  
        print('1')
        
        n_TCRA_Y_pred_ALL= list(range(len(TCRA_Y_pred_ALL)))
        n_TCRA_Y_acc_ALL= list(range(len(TCRA_Y_pred_ALL)))

        for k in range(len(TCRA_Y_pred_ALL)):
            
            if TCRA_Y_pred_ALL[k,0] == 0:
                n_TCRA_Y_pred_ALL[k] = 'True TCR-pMHC'
                n_TCRA_Y_acc_ALL[k] = 1 - TCRA_Y_pred_ALL_avg[k,0]
                
            if TCRA_Y_pred_ALL[k,0] == 1:
                n_TCRA_Y_pred_ALL[k] = 'False TCR-pMHC'
                n_TCRA_Y_acc_ALL[k] = 1 - TCRA_Y_pred_ALL_avg[k,0]                

        dataframe = pd.DataFrame({'TCRA_CDR3':TCRA_cdr3,'Epitope':Epitope,'Predict':n_TCRA_Y_pred_ALL,'Probability (predicted as a positive sample)':n_TCRA_Y_acc_ALL})
        print('2')
        csv_file = 'TCRA_pred.csv'
        csv_path = str(user_dir) + str(csv_file)
        dataframe.to_csv(csv_path,sep=',')
        print('3')

    elif user_select == 'B':
        print(user_select)
        Y_pred_FULL,Y_PRED_CNN,Y_PRED_RESNET=  pred_inte_all(user_dir,user_select)
        dataframe = pd.DataFrame({'CDR3B':TCRB_cdr3,'Epitope':Epitope,'y_true':y_true,'y_prob_FULL':Y_pred_FULL,'y_prob_CNN':Y_PRED_CNN,'y_prob_RESNET':Y_PRED_RESNET})
        dataframe.to_csv(save_path+'all_probability.csv')

        
        
    elif user_select == 'AB':
        print(user_select)
        
        TCRAB_Y_pred_ALL ,TCRAB_Y_acc_ALL=  pred_inte_all(user_dir,user_select)
        print('1')
        
        n_TCRAB_Y_pred_ALL= list(range(len(TCRAB_Y_pred_ALL)))
        n_TCRA_Y_acc_ALL= list(range(len(TCRAB_Y_pred_ALL)))
        n_TCRB_Y_acc_ALL= list(range(len(TCRAB_Y_pred_ALL)))
        
        for k in range(len(TCRAB_Y_pred_ALL)):
            if TCRAB_Y_pred_ALL[k] == 0:
                n_TCRAB_Y_pred_ALL[k] = 'True TCR-pMHC'
                
                n_TCRA_Y_acc_ALL[k] = 1 - TCRAB_Y_acc_ALL[0][k]
                n_TCRB_Y_acc_ALL[k] = 1 - TCRAB_Y_acc_ALL[1][k]               
                
            if TCRAB_Y_pred_ALL[k] == 1:
                n_TCRAB_Y_pred_ALL[k] = 'False TCR-pMHC'
                n_TCRA_Y_acc_ALL[k] = 1 - TCRAB_Y_acc_ALL[0][k]
                n_TCRB_Y_acc_ALL[k] = 1 - TCRAB_Y_acc_ALL[1][k] 
                
        print('2')
        dataframe = pd.DataFrame({'TCRA_CDR3':TCRA_cdr3,'TCRB_CDR3':TCRB_cdr3,'Epitope':Epitope,'Predict':n_TCRAB_Y_pred_ALL,
                                'Probability (TCRA_Epitope)':n_TCRA_Y_acc_ALL,'Probability (TCRB_Epitope)':n_TCRB_Y_acc_ALL})  
        csv_file = 'TCRAB_pred.csv'
        csv_path = str(user_dir) + str(csv_file)
        dataframe.to_csv(csv_path,sep=',')
        print('3')
        
def pred_inte_all(user_dir,user_select):
    if user_select == 'A':
        TCRA_Y_pred_ALL,TCRA_Y_pred_ALL_avg = pred_A(user_dir)
        return TCRA_Y_pred_ALL,TCRA_Y_pred_ALL_avg
    elif user_select == 'B':
        Y_pred_FULL,Y_PRED_CNN,Y_PRED_RESNET = pred_B(user_dir)
        return Y_pred_FULL,Y_PRED_CNN,Y_PRED_RESNET
    
    elif user_select == 'AB':    
        TCRA_Y_pred_ALL,TCRA_Y_pred_ALL_avg = pred_A(user_dir)
        TCRB_Y_pred_ALL,TCRB_Y_pred_ALL_avg = pred_B(user_dir)
        
        TCRAB_acc_ALL = [list(TCRA_Y_pred_ALL_avg[:,0]),list(TCRB_Y_pred_ALL_avg[:,0])]

        Y_pred_ALL = np.zeros(len(TCRA_Y_pred_ALL),)
        for i in range(len(TCRA_Y_pred_ALL)):
            if TCRA_Y_pred_ALL[i,0]+TCRB_Y_pred_ALL[i,0] > 0:  
                Y_pred_ALL[i] = 1
        return Y_pred_ALL,TCRAB_acc_ALL

    

2024-04-14 00:59:52.694604: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [4]:
input_file_path = "/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/less_data/predict_three_epitope.csv"
model_select = "B"  
data_path = "/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/data/repeat10/Prediction/three/less_data/"
save_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/evaluate/repeat10/Prediction/three/less_data/"
error_info,TCRB_cdr3,Epitope,y_true = deal_file(input_file_path, data_path, model_select)
output_file_path = save_outputfile(data_path,save_path, model_select, input_file_path,TCRB_cdr3,Epitope,y_true)

B


In [5]:
input_file_path = "/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/less_data/predict_four_epitope.csv"
model_select = "B"  
data_path = "/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/data/repeat10/Prediction/four/less_data/"
save_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/evaluate/repeat10/Prediction/four/less_data/"
error_info,TCRB_cdr3,Epitope,y_true = deal_file(input_file_path, data_path, model_select)
output_file_path = save_outputfile(data_path,save_path, model_select, input_file_path,TCRB_cdr3,Epitope,y_true)

B


In [6]:
four=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/evaluate/repeat10/Prediction/four/less_data/all_probability.csv")
four['Y_pred_FULL'] = four['y_prob_FULL'].apply(lambda x: 1 if x >= 0.5 else 0)
four['Y_pred_CNN'] = four['y_prob_CNN'].apply(lambda x: 1 if x >= 0.5 else 0)
four['Y_pred_RESNET'] = four['y_prob_RESNET'].apply(lambda x: 1 if x >= 0.5 else 0)
four.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/evaluate/repeat10/Prediction/four/less_data/all_probability.csv")
FULL=four[['CDR3B','Epitope','y_true','y_prob_FULL','Y_pred_FULL']]
new_columns = ['CDR3B', 'Epitope','y_true','y_prob','y_pred']
FULL.columns = new_columns
CNN=four[['CDR3B','Epitope','y_true','y_prob_CNN','Y_pred_CNN']]
CNN.columns = new_columns
RESNET=four[['CDR3B','Epitope','y_true','y_prob_RESNET','Y_pred_RESNET']]
RESNET.columns = new_columns
path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/evaluate/repeat10/Prediction/four/less_data/"
FULL.to_csv(path+'FULL_probability.csv')
CNN.to_csv(path+'CNN_probability.csv')
RESNET.to_csv(path+'RESNET_probability.csv')

In [7]:
three=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/evaluate/repeat10/Prediction/three/less_data/all_probability.csv")
three['Y_pred_FULL'] = three['y_prob_FULL'].apply(lambda x: 1 if x >= 0.5 else 0)
three['Y_pred_CNN'] = three['y_prob_CNN'].apply(lambda x: 1 if x >= 0.5 else 0)
three['Y_pred_RESNET'] = three['y_prob_RESNET'].apply(lambda x: 1 if x >= 0.5 else 0)
three.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/evaluate/repeat10/Prediction/three/less_data/all_probability.csv")
FULL=three[['CDR3B','Epitope','y_true','y_prob_FULL','Y_pred_FULL']]
new_columns = ['CDR3B', 'Epitope','y_true','y_prob','y_pred']
FULL.columns = new_columns
CNN=three[['CDR3B','Epitope','y_true','y_prob_CNN','Y_pred_CNN']]
CNN.columns = new_columns
RESNET=three[['CDR3B','Epitope','y_true','y_prob_RESNET','Y_pred_RESNET']]
RESNET.columns = new_columns
path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/evaluate/repeat10/Prediction/three/less_data/"
FULL.to_csv(path+'FULL_probability.csv')
CNN.to_csv(path+'CNN_probability.csv')
RESNET.to_csv(path+'RESNET_probability.csv')

In [9]:
name=['CNN','FULL','RESNET']
path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/evaluate/repeat10/Prediction/four/less_data/"
for i in name:
    ev(path,name)

In [10]:
name=['CNN','FULL','RESNET']
path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/evaluate/repeat10/Prediction/three/less_data/"
for i in name:
    ev(path,name)

# 0414修改

In [5]:
import pandas
import numpy as np
import os
os.chdir("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/")
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, matthews_corrcoef, confusion_matrix
from sklearn.metrics import f1_score,roc_auc_score,recall_score,precision_score

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint

from aaindexValues import aaindex1PCAValues

In [6]:
def pca_Validation(testfile_path,epi,name,save_path):
    def pca_code(seqs:list, row=30, n_features=16):
        aadict = aaindex1PCAValues(n_features)
        x = []
        col = n_features+1
        for i in range(len(seqs)):
            seq = seqs[i]
            n = len(seq)
            t = np.zeros(shape=(row, col))
            j = 0
            while j < n and j < row:
                t[j,:-1] = aadict[seq[j]]
                t[j,-1] = 0
                j += 1
            while j < row:
                t[j,-1] = 1
                j = j + 1
            x.append(t)
        return np.array(x)

    def read_seqs(file, model=1):
        data = pandas.read_csv(file)
        labels = data.Affinity
        cdr3 = data.CDR3B
        epitope = data.Epitope
        cdr3_seqs, epit_seqs = [], []
        for i in range(len(epitope)):
            if model == 1:
                cdr3_seqs.append(cdr3[i][2:-1])
            elif model == 2:
                cdr3_seqs.append(cdr3[i])
            epit_seqs.append(epitope[i])

        return cdr3_seqs, epit_seqs, labels

    def load_data(col=20, row=9, m=1):
        test_cdr3_seqs,  test_epit_seqs, test_labels = read_seqs(testFile, m)
        x_test = np.ndarray(shape=(len(test_cdr3_seqs), row, col+1, 2))
        x_test[:,:,:,0] = pca_code(test_cdr3_seqs, row, col)
        x_test[:,:,:,1] = pca_code(test_epit_seqs, row, col)

        y_test = np.array(test_labels)
        y_test = to_categorical(y_test, 2)
        return  (x_test, y_test)
    testFile = testfile_path + name+epi +'.csv'
   
    m=2
    row = 20
    for i in [10, 18, 20]:
        col = i 
        (x_test, y_test) = load_data(col=col, row=row, m=m)
        np.save(save_path + name+'_' +epi + 'TCRB_PCA{}_feature_array'.format(col), x_test)
        np.save(save_path + name+'_' +epi + 'TCRB_PCA{}_label_array'.format(col), y_test)


In [ ]:
name=['1','2','3','4','5','6']
me=['all','withCF']
epi=['_four','_three']
for i in name:
    for j in me:
        for k in epi:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/five/"+j+'/'
            save_path= "/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/data/repeat10/Prediction/repeat/"+j+'/'
            pca_Validation(testfile_path,k,i,save_path)


In [4]:
def ev (path,name):
    for file_name in name:
        df_data=pd.read_csv(path+file_name +'_probability.csv')
        metrics = []
        fpr_all = []
        tpr_all = []
        roc_auc_all = []
        for cls in df_data['Epitope'].unique():
            class_df = df_data[df_data['Epitope'] == cls]
            y_true = class_df['y_true']
            y_pred = class_df['y_pred']
            y_prob = class_df['y_prob']
            acc = accuracy_score(y_true, y_pred)
            auc_score = roc_auc_score(y_true, y_prob)if len(np.unique(y_true)) != 1 else 0
            recall = recall_score(y_true, y_pred)
            precision = precision_score(y_true, y_pred)  # Added precision calculation
            f1 = f1_score(y_true, y_pred)
            mcc = matthews_corrcoef(y_true, y_pred)
            metrics.append({
                'Class': cls,
                'ACC': acc,
                'AUC': auc_score,
                'Recall': recall,
                'Precision': precision,  # Added Precision metric
                'F1': f1,
                'MCC': mcc })
            from sklearn.metrics import roc_curve, auc 
            fpr, tpr, _ = roc_curve(y_true, y_prob)
            ALL_AUC = auc(fpr, tpr)
            fpr_all.append(fpr)
            tpr_all.append(tpr)
            roc_auc_all.append(ALL_AUC)
            metrics_class = pd.DataFrame(metrics)
            y_true_all = df_data['y_true']  # Modified from df to result
            y_pred_all = df_data['y_pred']  # Modified from df to result
            y_prob_all = df_data['y_prob']  # Modified from df to result
            acc_all = accuracy_score(y_true_all, y_pred_all)
            auc_all = roc_auc_score(y_true_all, y_prob_all)
            recall_all = recall_score(y_true_all, y_pred_all)
            precision_all = precision_score(y_true_all, y_pred_all)  # Added Precision calculation
            f1_all = f1_score(y_true_all, y_pred_all)
            mcc_all = matthews_corrcoef(y_true_all, y_pred_all)
            metrics_all = pd.DataFrame({
            'Metric': ['ACC', 'AUC', 'Recall', 'Precision', 'F1', 'MCC'],
            'Value': [acc_all, auc_all, recall_all, precision_all, f1_all, mcc_all]})
            all_value_row = pd.DataFrame({
            'Class': ['all_Value'],
            'ACC': [metrics_all.loc[0, 'Value']],
            'AUC': [metrics_all.loc[1, 'Value']],
            'Recall': [metrics_all.loc[2, 'Value']],
            'Precision': [metrics_all.loc[3, 'Value']],
            'F1': [metrics_all.loc[4, 'Value']],
            'MCC': [metrics_all.loc[5, 'Value']]
            })
            metrics_class = pd.concat([metrics_class, all_value_row], ignore_index=True)
            metrics_class.to_csv(path+file_name +'_all_result.csv')

In [5]:
from Model_Predict_Feature_Extraction import *

#from DLpTCR_server import *


import os
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.python.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.utils import plot_model
#from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint

from sklearn.utils import shuffle

from tensorflow.python.keras.models import load_model
import matplotlib.pyplot as plt
import csv
import pandas as pd

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 只显示error和warining信息 3 只显示error信息
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 这一行注释掉就是使用cpu，不注释就是使用gpu

import tensorflow.compat.v1 as tf

try:
    import tensorflow.python.keras as keras
except:
    import tensorflow.keras as keras

from tensorflow.python.keras import layers
from tensorflow.python.keras import backend as K

from sklearn.model_selection import KFold

from sklearn import metrics
from sklearn.metrics import accuracy_score,matthews_corrcoef,classification_report,confusion_matrix,precision_score,recall_score
from sklearn.metrics import f1_score,roc_auc_score, auc

from keras import regularizers

import os
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.python.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.utils import plot_model
#from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint

from sklearn.utils import shuffle

from tensorflow.python.keras.models import load_model
import matplotlib.pyplot as plt
import csv
import pandas as pd

    
def TCRB_Model_Integration(FULL_M,CNN_M,RESNET_M,FULL_Feature,CNN_Feature,RESNET_Feature):
    K.clear_session()
    tf.reset_default_graph()   
    
    FULL_model = load_model(FULL_M)
    CNN_model = load_model(CNN_M)
    RESNET_model = load_model(RESNET_M) 
    
    FULL_X = FULL_Feature
    FULL_X = FULL_X.reshape([len(FULL_X),20,19,2])

    CNN_X = CNN_Feature
    CNN_X = CNN_X.reshape([len(CNN_X),20,21,2])  
    
    RESNET_X = RESNET_Feature
    RESNET_X = RESNET_X.reshape([len(RESNET_X),20,11,2])

    Y_PRED_FULL = FULL_model.predict(FULL_X)
    Y_PRED_CNN = CNN_model.predict(CNN_X)
    Y_PRED_RESNET = RESNET_model.predict(RESNET_X)
    Y_pred_FULL=Y_PRED_FULL[:, 0]
    Y_PRED_CNN=Y_PRED_CNN[:, 0]
    Y_PRED_RESNET=Y_PRED_RESNET[:, 0]            

    del FULL_model
    del CNN_model
    del RESNET_model
    return Y_pred_FULL,Y_PRED_CNN,Y_PRED_RESNET
    
    
    
def pred_B(user_dir):
    TCRB_FULL_M = '../model/Article/FULL_B_ALL_pca18.h5'
    TCRB_CNN_M = '../model/Article/CNN_B_ALL_pca20.h5'
    TCRB_RESNET_M = '../model/Article/RESNET_B_ALL_pca10.h5'

    TCRB_FULL_Feature = np.load(str(user_dir) + "TCRB_PCA18_feature_array.npy")
    TCRB_CNN_Feature= np.load(str(user_dir) + "TCRB_PCA20_feature_array.npy")    
    TCRB_RESNET_Feature = np.load(str(user_dir) + "TCRB_PCA10_feature_array.npy") 
    
    Y_pred_FULL,Y_PRED_CNN,Y_PRED_RESNET = TCRB_Model_Integration(TCRB_FULL_M,TCRB_CNN_M,TCRB_RESNET_M,
                                              TCRB_FULL_Feature,TCRB_CNN_Feature,TCRB_RESNET_Feature)
    
    return Y_pred_FULL,Y_PRED_CNN,Y_PRED_RESNET



def save_outputfile(user_dir,save_path,user_select,excel_file_path,TCRB_cdr3,Epitope,y_true):
    if user_select == 'A':
        print(user_select)

        TCRA_Y_pred_ALL,TCRA_Y_pred_ALL_avg = pred_inte_all(user_dir,user_select)  
        print('1')
        
        n_TCRA_Y_pred_ALL= list(range(len(TCRA_Y_pred_ALL)))
        n_TCRA_Y_acc_ALL= list(range(len(TCRA_Y_pred_ALL)))

        for k in range(len(TCRA_Y_pred_ALL)):
            
            if TCRA_Y_pred_ALL[k,0] == 0:
                n_TCRA_Y_pred_ALL[k] = 'True TCR-pMHC'
                n_TCRA_Y_acc_ALL[k] = 1 - TCRA_Y_pred_ALL_avg[k,0]
                
            if TCRA_Y_pred_ALL[k,0] == 1:
                n_TCRA_Y_pred_ALL[k] = 'False TCR-pMHC'
                n_TCRA_Y_acc_ALL[k] = 1 - TCRA_Y_pred_ALL_avg[k,0]                

        dataframe = pd.DataFrame({'TCRA_CDR3':TCRA_cdr3,'Epitope':Epitope,'Predict':n_TCRA_Y_pred_ALL,'Probability (predicted as a positive sample)':n_TCRA_Y_acc_ALL})
        print('2')
        csv_file = 'TCRA_pred.csv'
        csv_path = str(user_dir) + str(csv_file)
        dataframe.to_csv(csv_path,sep=',')
        print('3')

    elif user_select == 'B':
        print(user_select)
        Y_pred_FULL,Y_PRED_CNN,Y_PRED_RESNET=  pred_inte_all(user_dir,user_select)
        dataframe = pd.DataFrame({'CDR3B':TCRB_cdr3,'Epitope':Epitope,'y_true':y_true,'y_prob_FULL':Y_pred_FULL,'y_prob_CNN':Y_PRED_CNN,'y_prob_RESNET':Y_PRED_RESNET})
        dataframe.to_csv(save_path+'all_probability.csv')

        
        
    elif user_select == 'AB':
        print(user_select)
        
        TCRAB_Y_pred_ALL ,TCRAB_Y_acc_ALL=  pred_inte_all(user_dir,user_select)
        print('1')
        
        n_TCRAB_Y_pred_ALL= list(range(len(TCRAB_Y_pred_ALL)))
        n_TCRA_Y_acc_ALL= list(range(len(TCRAB_Y_pred_ALL)))
        n_TCRB_Y_acc_ALL= list(range(len(TCRAB_Y_pred_ALL)))
        
        for k in range(len(TCRAB_Y_pred_ALL)):
            if TCRAB_Y_pred_ALL[k] == 0:
                n_TCRAB_Y_pred_ALL[k] = 'True TCR-pMHC'
                
                n_TCRA_Y_acc_ALL[k] = 1 - TCRAB_Y_acc_ALL[0][k]
                n_TCRB_Y_acc_ALL[k] = 1 - TCRAB_Y_acc_ALL[1][k]               
                
            if TCRAB_Y_pred_ALL[k] == 1:
                n_TCRAB_Y_pred_ALL[k] = 'False TCR-pMHC'
                n_TCRA_Y_acc_ALL[k] = 1 - TCRAB_Y_acc_ALL[0][k]
                n_TCRB_Y_acc_ALL[k] = 1 - TCRAB_Y_acc_ALL[1][k] 
                
        print('2')
        dataframe = pd.DataFrame({'TCRA_CDR3':TCRA_cdr3,'TCRB_CDR3':TCRB_cdr3,'Epitope':Epitope,'Predict':n_TCRAB_Y_pred_ALL,
                                'Probability (TCRA_Epitope)':n_TCRA_Y_acc_ALL,'Probability (TCRB_Epitope)':n_TCRB_Y_acc_ALL})  
        csv_file = 'TCRAB_pred.csv'
        csv_path = str(user_dir) + str(csv_file)
        dataframe.to_csv(csv_path,sep=',')
        print('3')
        
def pred_inte_all(user_dir,user_select):
    if user_select == 'A':
        TCRA_Y_pred_ALL,TCRA_Y_pred_ALL_avg = pred_A(user_dir)
        return TCRA_Y_pred_ALL,TCRA_Y_pred_ALL_avg
    elif user_select == 'B':
        Y_pred_FULL,Y_PRED_CNN,Y_PRED_RESNET = pred_B(user_dir)
        return Y_pred_FULL,Y_PRED_CNN,Y_PRED_RESNET
    
    elif user_select == 'AB':    
        TCRA_Y_pred_ALL,TCRA_Y_pred_ALL_avg = pred_A(user_dir)
        TCRB_Y_pred_ALL,TCRB_Y_pred_ALL_avg = pred_B(user_dir)
        
        TCRAB_acc_ALL = [list(TCRA_Y_pred_ALL_avg[:,0]),list(TCRB_Y_pred_ALL_avg[:,0])]

        Y_pred_ALL = np.zeros(len(TCRA_Y_pred_ALL),)
        for i in range(len(TCRA_Y_pred_ALL)):
            if TCRA_Y_pred_ALL[i,0]+TCRB_Y_pred_ALL[i,0] > 0:  
                Y_pred_ALL[i] = 1
        return Y_pred_ALL,TCRAB_acc_ALL

In [15]:
name=['1','2','3','4','5','6']
me=['all','withCF']
epi=['_four','_three']
for i in name:
    for j in me:
        for k in epi:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/five/"+j+'/'+i+k+'.csv'
            model_select = "B"  
            data_path = "/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/data/repeat10/Prediction/repeat/"+j+'/'+i+k
            save_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/evaluate/repeat10/Prediction/repeat/"+j+'/'+i+k
            error_info,TCRB_cdr3,Epitope,y_true = deal_file(testfile_path, data_path, model_select)
            output_file_path = save_outputfile(data_path,save_path, model_select, testfile_path,TCRB_cdr3,Epitope,y_true)
            four=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/evaluate/repeat10/Prediction/repeat/"+j+'/'+i+k+"all_probability.csv")
            four['Y_pred_FULL'] = four['y_prob_FULL'].apply(lambda x: 1 if x >= 0.5 else 0)
            four['Y_pred_CNN'] = four['y_prob_CNN'].apply(lambda x: 1 if x >= 0.5 else 0)
            four['Y_pred_RESNET'] = four['y_prob_RESNET'].apply(lambda x: 1 if x >= 0.5 else 0)
            four.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/evaluate/repeat10/Prediction/repeat/"+j+'/'+i+k+"all_probability.csv")
            four['Y_pred_FULL'] = four['y_prob_FULL'].apply(lambda x: 1 if x >= 0.5 else 0)
            FULL=four[['CDR3B','Epitope','y_true','y_prob_FULL','Y_pred_FULL']]
            new_columns = ['CDR3B', 'Epitope','y_true','y_prob','y_pred']
            FULL.columns = new_columns
            CNN=four[['CDR3B','Epitope','y_true','y_prob_CNN','Y_pred_CNN']]
            CNN.columns = new_columns
            RESNET=four[['CDR3B','Epitope','y_true','y_prob_RESNET','Y_pred_RESNET']]
            RESNET.columns = new_columns
            path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/evaluate/repeat10/Prediction/repeat/"+j+'/'+i+k
            four['Y_pred_FULL'] = four['y_prob_FULL'].apply(lambda x: 1 if x >= 0.5 else 0)
            FULL.to_csv(path+'FULL_probability.csv')
            CNN.to_csv(path+'CNN_probability.csv')
            RESNET.to_csv(path+'RESNET_probability.csv')
            name1=['CNN','FULL','RESNET']
            path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/evaluate/repeat10/Prediction/repeat/"+j+'/'+i+k
            for l in name1:
                ev(path,name1)


/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/five/all/1_four.csv
B
/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/five/all/1_three.csv
B
/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/five/withCF/1_four.csv
B
/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/five/withCF/1_three.csv
B
/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/five/all/2_four.csv
B
/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/five/all/2_three.csv
B
/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross